## Airbnb (New York City)
- 미국 NYC Airbnb 목록(2019)
- 데이터 출처:https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data (License CC0: Public Domain)
- 프로젝트 목적: 가격 예측(price)
- 제공 데이터(3개): train.csv, test.csv, y_test(최종 채점용)
- 평가 방식: MSE


## Baseline

In [20]:
# 라이브러리 
import pandas as pd

### 데이터 불러오기

In [21]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

### 데이터 전처리

In [22]:
# 결측치 컬럼 삭제 (last_review)
train = train.drop(columns=["last_review"])
test = test.drop(columns=["last_review"])

In [23]:
# 결측치 채우기
train["reviews_per_month"] = train["reviews_per_month"].fillna(0)
test["reviews_per_month"] = test["reviews_per_month"].fillna(0)

In [24]:
train.isnull().sum()

id                                 0
name                              12
host_id                            0
host_name                         17
neighbourhood_group                0
neighbourhood                      0
latitude                           0
longitude                          0
room_type                          0
price                              0
minimum_nights                     0
number_of_reviews                  0
reviews_per_month                  0
calculated_host_listings_count     0
availability_365                   0
dtype: int64

In [25]:
# 가격 값 복사
target = train["price"]
train = train.drop(columns=["price"])

In [26]:
# 수치형 피쳐 선택
cols = [
    "host_id",
    "latitude",
    "longitude",
    "minimum_nights",
    "number_of_reviews",
    "reviews_per_month",
    "calculated_host_listings_count",
    "availability_365",
]

train = train[cols]
test = test[cols]

### 검증 데이터 분리

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=2022)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((31292, 8), (7824, 8), (31292,), (7824,))

### 머신러닝

In [28]:
# Xgboost
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

model = XGBRegressor(random_state=2022)

model.fit(X_train, y_train)
pred = model.predict(X_val)
mean_squared_error(y_val, pred)

33378.923185220534

### 채점

In [29]:
# test 데이터 예측 및 평가
y_test = pd.read_csv("./data/y_test.csv")

#Xgboost
pred = model.predict(test)
mean_squared_error(y_test, pred)

54244.32571482477